## Import

## Exploring data

### Read raw data from file

### How many rows and how many columns does the raw data have?

### What is the meaning of each row?

### Are there duplicated rows?

### What is the meaning of each column?

### What is the current data type of each column? Are there columns having inappropriate data types?


### With each numerical column, how are values distributed?

### With each categorical column, how are values distributed?